In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import glob
import logging
logging.basicConfig(filename="./sales_table_log.log", level=logging.INFO)

def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)
            
last_sturday_ty = datetime.date(2020,10,31)
first_week_ty=datetime.date(2020,8,2)

last_sturday_ly = last_sturday_ty-datetime.timedelta(days=52*7)
first_week_ly=first_week_ty-datetime.timedelta(days=52*7)

df_prod_mapping_2020=pd.read_csv("/mnt/clients/juba/hqjubaapp02/sharefolder/biglots_data/static_files/ProductTaxonomy/MediaStormProductTaxonomy20201101-135215-182.txt",
                            sep="|",dtype=str,usecols=['division_id','department_id','class_code_id','subclass_id'])
df_prod_mapping_2019=pd.read_csv("/mnt/clients/juba/hqjubaapp02/sharefolder/biglots_data/static_files/ProductTaxonomy/MediaStormProductTaxonomy20190101-135843-066.txt",
                            sep="|",dtype=str,usecols=['division_id','department_id','class_code_id','subclass_id'])


In [2]:
last_sturday_ty,first_week_ty,last_sturday_ly,first_week_ly

(datetime.date(2020, 10, 31),
 datetime.date(2020, 8, 2),
 datetime.date(2019, 11, 2),
 datetime.date(2019, 8, 4))

In [3]:
list_POS_this_year=list(recursive_file_gen("/mnt/clients/juba/hqjubaapp02/sharefolder/biglots_data/2020_by_weeks/"))
list_POS_this_year=[x for x in list_POS_this_year if "daily" in x.lower()]

list_POS_this_year=[x for x in list_POS_this_year if x.split("/MediaStorm_")[1][:10]>=str(first_week_ty)]
list_POS_this_year=[x for x in list_POS_this_year if x.split("/MediaStorm_")[1][:10]<=str(last_sturday_ty)]
list_POS_this_year.sort()
# 
list_POS_last_year=list(recursive_file_gen("/mnt/clients/juba/hqjubaapp02/sharefolder/biglots_data/2019_by_weeks/"))
list_POS_last_year=[x for x in list_POS_last_year if "daily" in x.lower()]
list_POS_last_year.sort()


list_POS_last_year=[x for x in list_POS_last_year if x.split("/MediaStorm_")[1][:10]>=str(first_week_ly)]
list_POS_last_year=[x for x in list_POS_last_year if x.split("/MediaStorm_")[1][:10]<=str(last_sturday_ly)]

print(len(list_POS_this_year),len(list_POS_last_year))


13 13


In [4]:
df_2020=pd.DataFrame()

for file in list_POS_this_year:
    df=pd.read_csv(file,dtype=str,sep="|")
    df=pd.merge(df,df_prod_mapping_2020,on=['class_code_id','subclass_id'],how="left")
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df['division_id']=df['division_id'].fillna("-1")
    df['department_id']=df['department_id'].fillna("-1")
    df['rewards_label']=np.where(pd.isnull(df['customer_id_hashed']),"non_rewards","rewards")
    df['store_label']=np.where(df['location_id']=="6990","online","instore")
    week_end_dt=df['transaction_dt'].max()
    
    df_sales=df.groupby(['division_id','department_id','rewards_label','store_label'])['item_transaction_amt'].sum().to_frame().reset_index()
    
    df_trans=df[['location_id','transaction_dt','transaction_id','rewards_label','division_id','department_id','store_label']].drop_duplicates()
    df_trans=df_trans.groupby(['division_id','department_id','rewards_label','store_label'])['transaction_id'].count().to_frame().reset_index()
    df=pd.merge(df_sales,df_trans,on=['division_id','department_id','rewards_label','store_label'],how="outer")
    
    df['week_end_dt']=week_end_dt
    df=df.rename(columns={"item_transaction_amt":"sales","transaction_id":"trans"})
    df_2020=df_2020.append(df)
    print(week_end_dt,datetime.datetime.now())
    

2020-08-08 2020-11-06 16:52:58.094911
2020-08-15 2020-11-06 16:54:00.735835
2020-08-22 2020-11-06 16:55:03.724487
2020-08-29 2020-11-06 16:56:06.286853
2020-09-05 2020-11-06 16:57:10.871204
2020-09-12 2020-11-06 16:58:13.590284
2020-09-19 2020-11-06 16:59:15.083438
2020-09-26 2020-11-06 17:00:20.802142
2020-10-03 2020-11-06 17:01:29.286193
2020-10-10 2020-11-06 17:02:33.501766
2020-10-17 2020-11-06 17:03:40.332029
2020-10-24 2020-11-06 17:04:45.996174
2020-10-31 2020-11-06 17:05:57.053926


In [5]:
df_2019=pd.DataFrame()
df_prod_mapping=pd.read_csv("/mnt/clients/juba/hqjubaapp02/sharefolder/biglots_data/static_files/ProductTaxonomy/MediaStormProductTaxonomy20190401-134939-109.txt",
                            sep="|",dtype=str,usecols=['division_id','department_id','class_code_id','subclass_id'])
for file in list_POS_last_year:
    df=pd.read_csv(file,dtype=str,sep="|")
    df=pd.merge(df,df_prod_mapping_2019,on=['class_code_id','subclass_id'],how="left")
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df['division_id']=df['division_id'].fillna("-1")
    df['department_id']=df['department_id'].fillna("-1")
    df['rewards_label']=np.where(pd.isnull(df['customer_id_hashed']),"non_rewards","rewards")
    df['store_label']=np.where(df['location_id']=="6990","online","instore")
    week_end_dt=df['transaction_dt'].max()
    
    df_sales=df.groupby(['division_id','department_id','rewards_label','store_label'])['item_transaction_amt'].sum().to_frame().reset_index()
    
    df_trans=df[['location_id','transaction_dt','transaction_id','rewards_label','division_id','department_id','store_label']].drop_duplicates()
    df_trans=df_trans.groupby(['division_id','department_id','rewards_label','store_label'])['transaction_id'].count().to_frame().reset_index()
    df=pd.merge(df_sales,df_trans,on=['division_id','department_id','rewards_label','store_label'],how="outer")
    
    df['week_end_dt']=week_end_dt
    df=df.rename(columns={"item_transaction_amt":"sales","transaction_id":"trans"})
    df_2019=df_2019.append(df)
    print(week_end_dt,datetime.datetime.now())
    

2019-08-10 2020-11-06 17:06:55.343932
2019-08-17 2020-11-06 17:07:55.686696
2019-08-24 2020-11-06 17:08:58.046712
2019-08-31 2020-11-06 17:10:02.248225
2019-09-07 2020-11-06 17:11:00.402453
2019-09-14 2020-11-06 17:11:58.493235
2019-09-21 2020-11-06 17:12:57.106603
2019-09-28 2020-11-06 17:14:02.841710
2019-10-05 2020-11-06 17:15:10.373700
2019-10-12 2020-11-06 17:16:03.396261
2019-10-19 2020-11-06 17:17:12.028448
2019-10-26 2020-11-06 17:18:07.453396
2019-11-02 2020-11-06 17:19:07.677454


In [6]:
print("df_2019.shape",df_2019.shape)
print("df_2020.shape",df_2020.shape)

df_2019['division_id']=df_2019['division_id'].astype(int)
df_2020['division_id']=df_2020['division_id'].astype(int)
df_2019['department_id']=df_2019['department_id'].astype(int)
df_2020['department_id']=df_2020['department_id'].astype(int)

df_2019=df_2019.sort_values(['division_id','department_id','rewards_label','store_label'])
df_2020=df_2020.sort_values(['division_id','department_id','rewards_label','store_label'])


df_2019.shape (2129, 7)
df_2020.shape (2180, 7)


In [7]:
df_2019_wide=df_2019.pivot_table(index=['division_id','department_id','rewards_label','store_label'],columns="week_end_dt",values="sales").reset_index()
df_2020_wide=df_2020.pivot_table(index=['division_id','department_id','rewards_label','store_label'],columns="week_end_dt",values="sales").reset_index()
df_2019_wide.insert(0,'year',"2019")
df_2020_wide.insert(0,'year',"2020")

In [8]:
for col in df_2019_wide.columns.tolist():
    if "-" in col:
        df_2019_wide[col]=df_2019_wide[col].fillna(0)
        
for col in df_2020_wide.columns.tolist():
    if "-" in col:
        df_2020_wide[col]=df_2020_wide[col].fillna(0)

In [9]:
division_name=pd.read_table("/mnt/clients/juba/hqjubaapp02/sharefolder/biglots_data/static_files/MediaStorm Data Extract - Division Names.txt",sep="|")
department_name=pd.read_table("/mnt/clients/juba/hqjubaapp02/sharefolder/biglots_data/static_files/MediaStorm Data Extract - Department Names.txt",sep="|")


In [10]:
writer=pd.ExcelWriter("./BL_sales_by_department_week_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_2019_wide.to_excel(writer,"2019_wide_all",index=False)
df_2020_wide.to_excel(writer,"2020_wide_all",index=False)
division_name.to_excel(writer,"division_name",index=False)
department_name.to_excel(writer,"department_name",index=False)
df_2019.to_excel(writer,"2019_long_all",index=False)
df_2020.to_excel(writer,"2020_long_all",index=False)
for comb,group in df_2019_wide.groupby(['rewards_label','store_label']):
    year=group['year'].unique().tolist()[0]
    tab_name="_".join(comb)+"_"+year
    group.to_excel(writer,tab_name,index=False)
    
for comb,group in df_2020_wide.groupby(['rewards_label','store_label']):
    year=group['year'].unique().tolist()[0]
    tab_name="_".join(comb)+"_"+year
    group.to_excel(writer,tab_name,index=False)
writer.save()